# Learn-Function-calling-Enhanced

参考文档：[https://platform.openai.com/docs/guides/function-calling](https://platform.openai.com/docs/guides/function-calling)

参考API：[https://platform.openai.com/docs/api-reference/chat/create](https://platform.openai.com/docs/api-reference/chat/create)

## 增强特性
- 保持原有Function Calling逻辑
- 添加本地模型集成示例
- Python 3.11优化

# Function calling 函数调用 - 增强版

Learn how to connect large language models to external tools.

了解如何将大型语言模型连接到外部工具。

Example invoking multiple function calls in one response

在一个响应中调用多个函数调用的示例

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())



# 检查DeepSeek API Key
if os.getenv('DEEPSEEK_API_KEY'):
    print("✅ DeepSeek API Key 已配置")
    print("🚀 Function Calling使用DeepSeek模型")
else:
    print("❌ 请在.env文件中配置DEEPSEEK_API_KEY")
    raise ValueError("DeepSeek API Key未配置")

✅ DeepSeek API Key 已配置
🚀 Function Calling使用DeepSeek模型


In [2]:
from openai import OpenAI
import json
import os

# 使用DeepSeek API
client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com"
)

print("🧠 DeepSeek客户端初始化完成")
print("🇨🇳 支持Function Calling功能")

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"): 
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    elif "beijing" in location.lower() or "北京" in location:
        return json.dumps({"location": "北京", "temperature": "15", "unit": unit})
    elif "shanghai" in location.lower() or "上海" in location:
        return json.dumps({"location": "上海", "temperature": "25", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

🧠 DeepSeek客户端初始化完成
🇨🇳 支持Function Calling功能


In [3]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, Beijing, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    
    # 使用DeepSeek模型进行Function Calling
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    print("=== 第一次DeepSeek响应 ===")
    print(response)
    response_message = response.choices[0].message
    print("\n=== 响应消息 ===")
    print(response_message)
    tool_calls = response_message.tool_calls
    print("\n=== 工具调用 ===")
    print(tool_calls)
    
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            print(f"\n🔧 调用函数 {function_name}，参数: {function_args}，结果: {function_response}")
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="deepseek-chat",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response

print("🚀 DeepSeek Function Calling函数定义完成")

🚀 DeepSeek Function Calling函数定义完成


## Function Calling 实现说明

这段代码演示了如何使用DeepSeek模型进行Function Calling，让我逐步解释代码：

**核心组件**：
1. **DeepSeek客户端**: 通过配置API Key和base_url连接到DeepSeek服务
2. **工具函数**: get_current_weather函数模拟外部API调用
3. **对话处理**: run_conversation()函数负责整个Function Calling流程

**执行流程**：
a. 首先，构造包含用户消息和可用功能的消息列表和工具列表
b. 调用client.chat.completions.create()发送对话和工具信息给DeepSeek模型
c. DeepSeek模型分析是否需要调用函数，tool_choice="auto"表示自动选择
d. 如果需要调用函数，根据返回信息调用相应函数并获取结果
e. 将函数结果添加到对话中，再次发送给DeepSeek模型获取最终回复

**技术优势**：
- 🇨🇳 DeepSeek对中文理解更好，Function Calling更准确
- 💰 成本比OpenAI GPT模型低很多
- 🔧 支持复杂的工具调用和多轮对话
- 🚀 可以与本地BGE模型结合，实现完全本土化的AI应用

In [4]:
result = run_conversation()
print("\n=== 最终结果 ===")
print(result)

=== 第一次DeepSeek响应 ===
ChatCompletion(id='e3caac0e-02d7-42e6-9fa4-5b4319914adb', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_4fd29ac6-0d99-44b7-bc42-b6fb41252103', function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), type='function', index=0), ChatCompletionMessageToolCall(id='call_1_5567322e-18f1-4c63-ab8c-3f7de7578b64', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function', index=1), ChatCompletionMessageToolCall(id='call_2_21c495fc-592c-4eac-90fb-c9af377686b5', function=Function(arguments='{"location": "Beijing"}', name='get_current_weather'), type='function', index=2), ChatCompletionMessageToolCall(id='call_3_b4653b24-e82e-49ec-a1d9-c8ebdb5bf929', function=Function(arguments='{"location": "Paris"}', na

In [5]:
print("=== 最终回答 ===")
print(result.choices[0].message.content)

=== 最终回答 ===
Here's the current weather in each city:  

- **San Francisco**: 72°F (temperature unit not specified, but likely Fahrenheit)  
- **Tokyo**: 10°C (temperature unit not specified, but likely Celsius)  
- **Beijing (北京)**: 15°C (temperature unit not specified, but likely Celsius)  
- **Paris**: 22°C (temperature unit not specified, but likely Celsius)  

Let me know if you'd like more details!


## 增强示例：结合本地模型的Function Calling

In [6]:
# 演示如何将Function Calling与本地嵌入模型结合
from sentence_transformers import SentenceTransformer
import numpy as np
import os

# 使用环境变量配置的BGE模型路径
model_path = os.getenv('BGE_MODEL_PATH', "BAAI/bge-small-zh-v1.5")
print(f"📁 使用BGE模型路径: {model_path}")

try:
    local_model = SentenceTransformer(model_path)
    print("✅ BGE本地模型加载成功")
except Exception as e:
    print(f"⚠️ BGE模型加载失败: {e}")
    print("💡 将使用在线模型作为备选")
    local_model = SentenceTransformer("BAAI/bge-small-zh-v1.5")

def semantic_search_function(query, documents):
    """使用本地BGE模型进行语义搜索"""
    # 对查询进行编码
    query_embedding = local_model.encode([query])
    
    # 对文档进行编码
    doc_embeddings = local_model.encode(documents)
    
    # 计算相似度
    similarities = np.dot(query_embedding, doc_embeddings.T)[0]
    
    # 返回最相似的文档
    best_idx = np.argmax(similarities)
    return {
        "best_document": documents[best_idx],
        "similarity_score": float(similarities[best_idx]),
        "index": int(best_idx)
    }

# 测试本地语义搜索
test_docs = [
    "Python是一门编程语言",
    "机器学习是人工智能的分支", 
    "LangChain是用于构建AI应用的框架",
    "Qdrant是向量数据库",
    "DeepSeek是国产大语言模型"
]

result = semantic_search_function("什么是AI框架", test_docs)
print(f"🔍 搜索结果: {result}")

print("✅ DeepSeek + BGE本地模型组合测试完成")

C:\study_software2\Anaconda\envs\py311_MockInterview\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📁 使用BGE模型路径: C:\Users\guohaoyu\.cache\huggingface\hub\models--BAAI--bge-small-zh-v1.5\snapshots\7999e1d3359715c523056ef9478215996d62a620
✅ BGE本地模型加载成功
🔍 搜索结果: {'best_document': 'LangChain是用于构建AI应用的框架', 'similarity_score': 0.7385997772216797, 'index': 2}
✅ DeepSeek + BGE本地模型组合测试完成
